In [3]:
import json
import re

### 清洗

In [72]:
# 输入和输出路径
input_path = "D:\Work\讯飞\上海政策法规\上海政策法规\zxzdgz.json"
output_path = "D:\Work\讯飞\上海政策法规\上海政策法规\zxzdgz_cleaned.json"

# 文本预处理函数
def clean_text(raw_text):
    cleaned = raw_text
    # 修复“第\n六\n条” → “第六条”
    # cleaned = re.sub(r'第\s*\n*\s*([一二三四五六七八九十百千万\d]+)\s*\n*\s*条', r'第\1条', raw_text)
    # 将段内换行统一为空格
    # cleaned = re.sub(r'([^\n])\n([^\n])', r'\1 \2', cleaned)
    # 去掉连续空行
    # cleaned = re.sub(r'\n{2,}', '\n', cleaned)
    # 去掉页眉页脚类模板信息（可按具体情况调整）
    # cleaned = re.sub(r'(信息来源：.*?$)', '', cleaned, flags=re.MULTILINE)
    cleaned = cleaned.replace('\n', '').replace('\r', '').replace('\t', '').replace('\xa0', '')
    return cleaned.strip()

# 读取原始 JSON
with open(input_path, "r", encoding="utf-8") as f:
    documents = json.load(f)

# 对每条记录处理 content 字段
for entry in documents:
    if "content" in entry:
        entry["content"] = clean_text(entry["content"])

# 保存处理后的 JSON
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(documents, f, ensure_ascii=False, indent=2)



### 切分

In [184]:
import re
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client["xunfei"]
gj = db["国家政策法规"]
sh = db["上海政策汇编"]
zx = db["中心制度规则"]
source_col = gj     # e.g. 原始数据，如“policies”
target_col = db["国家按章分"]          # 切分后的目标集合

In [13]:
def smart_split(text):
    """
    支持以下条目格式的切分：
    - 第X条
    - 一、二、三、
    - （一）（二）
    - 1）、2）
    """
    # 正则模式匹配多种编号形式（保留分隔符）
    pattern = r'(?=(第[一二三四五六七八九十百千万\d]+条|' \
              r'第[一二三四五六七八九十百千万\d]+节|' \
              r'[一二三四五六七八九十百千万]+、|' \
              r'\d+[）.]))'
              # r'（[一二三四五六七八九十]+）|' \
    # pattern = r'(第[一二三四五六七八九十百千万\d]=+章' \
    #           r'[一二三四五六七八九十百千万]+、)'
    
    parts = re.split(pattern, text)
    chunks = []
    title_st = '前言'
    body_st = parts[0].strip()
    chunks.append((title_st, body_st))
    # print(chunks)
    for i in range(1, len(parts)-1, 2):
        title = parts[i].strip()
        body = parts[i+1].strip()
        if len(body) > 20:
            chunks.append((title, body))
    return chunks

In [127]:
import re

def split_sub_article(text):
    """对超长文本进行（一级）按（（一）（二））编号切分"""
    pattern_sub = r'(?=(（[一二三四五六七八九十]+）' \
              r'|\d+[）.]' \
              r'|\d+))'

    parts = re.split(pattern_sub, text)
    result = []
    if parts:
        result.append(parts[0].strip())  # 第一个前导段
    for i in range(1, len(parts)):
        result.append("（" + parts[i].strip())
    return result

def smart_split(text, max_length=1500):
    """
    支持以下条目格式的切分：
    - 第X条、节
    - 一、二、三、
    - 1）、2）
    - 超过8192字符的块再细分为（一级）条目如：（一）（二）等
    """
    # 一级大条目正则（保留分隔符）
    pattern = r'(?=(第[一二三四五六七八九十百千万\d]+条|' \
              r'第[一二三四五六七八九十百千万\d]+节|' \
              r'[一二三四五六七八九十百千万]+、))' 

    parts = re.split(pattern, text)
    chunks = []

    # 前言部分
    if parts:
        title_st = '前言'
        body_st = parts[0].strip()
        if body_st:
            chunks.append((title_st, body_st))

    # 从第一个标题开始循环
    for i in range(1, len(parts), 2):
        title = parts[i].strip()
        body = parts[i + 1].strip() if i + 1 < len(parts) else ''
        if len(body) > max_length:
            print(body)
            sub_bodies = split_sub_article(body)
            for j, sub_body in enumerate(sub_bodies):
                sub_title = f"{title}（子段{j+1}）"
                chunks.append((sub_title, sub_body))
        else:
            chunks.append((title, body))

    return chunks


In [182]:
import re

def split_sub_article(text):
    """第二层切分：对超长文本进行按（（一）（二））编号细分"""
    pattern_sub = r'(?=（[一二三四五六七八九十]+）|\d{1,2}[）.])'
    parts = re.split(pattern_sub, text)
    result = []
    if parts:
        result.append(parts[0].strip())  # 前导段
    for i in range(1, len(parts)):
        result.append("（" + parts[i].strip())
    return result

def split_long_text(text, max_len=1000, overlap=200):
    """第三层切分：对依然很长的文本按长度切分，并支持overlap"""
    result = []
    start = 0
    text_len = len(text)
    while start < text_len:
        end = min(start + max_len, text_len)
        chunk = text[start:end].strip()
        result.append(chunk)
        if end == text_len:
            break
        start = end - overlap  # 注意回退overlap
    return result

def smart_split(text, max_article_len=8192, max_final_len=1000, overlap=200):
    """
    三层切分逻辑：
    1. 第X条、节；一、二、三；1）、2）为主标题切
    2. 超过8192的部分用（一级）（二级）编号再切
    3. 超过1000再按字符长度切分，带overlap
    """
    # 一级大条目正则
    pattern = r'(?=(第[一二三四五六七八九十百千万\d]+条|' \
              r'第[一二三四五六七八九十百千万\d]+节|' \
              r'[一二三四五六七八九十百千万]+、))' 
    parts = re.split(pattern, text)
    chunks = []

    # 前言
    if parts:
        title_st = '前言'
        body_st = parts[0].strip()
        sub_sections = [body_st]
        if len(body_st) > max_article_len:
            sub_sections = split_sub_article(body_st)
        for m, sub_body in enumerate(sub_sections):
            if len(sub_body) > max_final_len:
                split_parts = split_long_text(sub_body, max_len=max_final_len, overlap=overlap)
                for n, part in enumerate(split_parts):
                    sub_title = f"{title_st}（子段{m+1}-{n+1}）"
                    chunks.append((sub_title, part))
            else:
                sub_title = f"{title_st}（子段{m+1}）" if len(sub_sections) > 1 else title_st
                chunks.append((sub_title, sub_body))
        # if body_st:
        #     chunks.append((title_st, body_st))

    for i in range(1, len(parts), 2):
        title = parts[i].strip()
        body = parts[i + 1].strip() if i + 1 < len(parts) else ''

        sub_sections = [body]

        if len(body) > max_article_len:
            sub_sections = split_sub_article(body)

        for j, sub_body in enumerate(sub_sections):
            if len(sub_body) > max_final_len:
                split_parts = split_long_text(sub_body, max_len=max_final_len, overlap=overlap)
                for k, part in enumerate(split_parts):
                    sub_title = f"{title}（子段{j+1}-{k+1}）"
                    chunks.append((sub_title, part))
            else:
                sub_title = f"{title}（子段{j+1}）" if len(sub_sections) > 1 else title
                chunks.append((sub_title, sub_body))

    return chunks


In [185]:
j = 1
for doc in source_col.find():
    doc_id = doc.get("relative_path", "").split("/")[-1]
    content = doc.get("content", "")
    title = doc.get("title", "")
    url = doc.get("full_url", "")
    if len(content) > 1000:   
        para_list = smart_split(content)
    else:
        para_list = [("全文", content)]
    inserts = []
    for idx, (article_title, body) in enumerate(para_list):
        para_id = f"{doc_id}_T{idx+1:02d}"
        inserts.append({
            "para_id": para_id,
            "doc_id": doc_id,
            "article": article_title,
            "text": f"{article_title} {body}",
            "source_title": title,
            "source_url": url,
            "vectorized": False
        })

    if inserts:
        target_col.insert_many(inserts)
        print(f"✅ 插入 {len(inserts)} 条 from {doc_id}")
        print(j)
        j += 1
    else:
        print(doc_id)


✅ 插入 18 条 from 8063264
1
✅ 插入 109 条 from 8063251
2
✅ 插入 44 条 from 8063270
3
✅ 插入 74 条 from 8063254
4
✅ 插入 46 条 from 8063271
5
✅ 插入 70 条 from 8063277
6
✅ 插入 46 条 from 8063262
7
✅ 插入 57 条 from 8063261
8
✅ 插入 32 条 from 8063273
9
✅ 插入 52 条 from 8063274
10
✅ 插入 61 条 from 8063249
11
✅ 插入 7 条 from 8063501
12
✅ 插入 25 条 from 8063500
13
✅ 插入 30 条 from 8063275
14
✅ 插入 12 条 from 8063276
15
✅ 插入 11 条 from 8063503
16
✅ 插入 5 条 from 8063504
17
✅ 插入 39 条 from 8063288
18
✅ 插入 6 条 from 8063523
19
✅ 插入 63 条 from 8063285
20
✅ 插入 87 条 from 8063289
21
✅ 插入 27 条 from 8063278
22
✅ 插入 42 条 from 8063287
23
✅ 插入 28 条 from 8063290
24
✅ 插入 21 条 from 8063294
25
✅ 插入 90 条 from 8063293
26
✅ 插入 38 条 from 8063344
27
✅ 插入 52 条 from 8063454
28
✅ 插入 24 条 from 8063452
29
✅ 插入 98 条 from 8063292
30
✅ 插入 81 条 from 8063457
31
✅ 插入 102 条 from 8063455
32
✅ 插入 48 条 from 8063459
33
✅ 插入 29 条 from 8063461
34
✅ 插入 47 条 from 8063460
35
✅ 插入 22 条 from 8063480
36
✅ 插入 96 条 from 8063481
37
✅ 插入 57 条 from 8063482
38
✅ 插入 21 条 from 8063486

In [151]:
for i in range(len(inserts)):
    print(len(inserts[i].get("text")))

338


In [159]:
from pymongo import MongoClient

col_mongo_name = "国家政策法规_split"
mongo = MongoClient("mongodb://localhost:27017/")
db = mongo["xunfei"]
col_mongo = db[col_mongo_name]

In [160]:
docs = col_mongo.find()

In [161]:
docs

In [162]:
texts = []
for doc in docs:
    texts .append(doc["text"])

In [163]:
print(len(max(texts, key=len)))

10470


In [164]:
print(max(texts, key=len))

1. 2万亿元。--乡村新型服务业类型丰富。农林牧渔专业及辅助性活动产值达到1万亿元，农产品网络销售额达到1万亿元。--农村创新创业更加活跃。返乡入乡创新创业人员超过1500万人。第三章　提升农产品加工业农产品加工业是国民经济的重要产业。农产品加工业从种养业延伸出来，是提升农产品附加值的关键，也是构建农业产业链的核心。进一步优化结构布局，培育壮大经营主体，提升质量效益和竞争力。第一节　完善产业结构统筹发展农产品初加工、精深加工和综合利用加工，推进农产品多元化开发、多层次利用、多环节增值。拓展农产品初加工。鼓励和支持农民合作社、家庭农场和中小微企业等发展农产品产地初加工，减少产后损失，延长供应时间，提高质量效益。果蔬、奶类、畜禽及水产品等鲜活农产品，重点发展预冷、保鲜、冷冻、清洗、分级、分割、包装等仓储设施和商品化处理，实现减损增效。粮食等耐储农产品，重点发展烘干、储藏、脱壳、去杂、磨制等初加工，实现保值增值。食用类初级农产品，重点发展发酵、压榨、灌制、炸制、干制、腌制、熟制等初加工，满足市场多样化需求。棉麻丝、木竹藤棕草等非食用类农产品，重点发展整理、切割、粉碎、打磨、烘干、拉丝、编织等初加工，开发多种用途。提升农产品精深加工。引导大型农业企业加快生物、工程、环保、信息等技术集成应用，促进农产品多次加工，实现多次增值。发展精细加工，推进新型非热加工、新型杀菌、高效分离、清洁生产、智能控制、形态识别、自动分选等技术升级，利用专用原料，配套专用设备，研制专用配方，开发类别多样、营养健康、方便快捷的系列化产品。推进深度开发，创新超临界萃取、超微粉碎、生物发酵、蛋白质改性等技术，提取营养因子、功能成分和活性物质，开发系列化的加工制品。推进综合利用加工。鼓励大型农业企业和农产品加工园区推进加工副产物循环利用、全值利用、梯次利用，实现变废为宝、化害为利。采取先进的提取、分离与制备技术，推进稻壳米糠、麦麸、油料饼粕、果蔬皮渣、畜禽皮毛骨血、水产品皮骨内脏等副产物综合利用，开发新能源、新材料等新产品，提升增值空间。第二节　优化空间布局按照“粮头食尾”“农头工尾”要求，统筹产地、销区和园区布局，形成生产与加工、产品与市场、企业与农户协调发展的格局。推进农产品加工向产地下沉。向优势区域聚集，引导大型农业企业重心下沉，在粮食生产功能区、重要农产品保护区、特色农产品优势区和水产品主产区，建

In [186]:
gjazf = db["国家按章分"]
sh = db["上海政策汇编_split"]

In [187]:
gjazf_docs = gjazf.find()
sh_docs = sh.find()
gjazf_texts, sh_texts = [] , []
for doc in gjazf_docs:
    gjazf_texts.append(doc["text"])
for doc in sh_docs:
    sh_texts.append(doc["text"])

In [188]:
print(len(max(sh_texts, key=len)))
print(len(max(gjazf_texts, key=len)))

1327
1012


In [189]:
max(gjazf_texts, key=len)

'前言（子段122-1） （4）挂牌主持人连续三次宣布同一应价或报价而没有人再应价或出价，且该价格不低于底价的，挂牌主持人落槌表示现场竞价成交，宣布最高应价或报价者为竞得人。成交结果对竞得人和出让人均具有法律效力。最高应价或报价低于底价的，挂牌主持人宣布现场竞价终止。在现场竞价中无人参加竞买或无人应价或出价的，以挂牌截止时出价最高者为竞得人，但低于挂牌出让底价者除外。13．6\u3000签订《成交确认书》确定竞得人后，挂牌人与竞得人当场签订《成交确认书》。挂牌人或竞得人不按规定签订《成交确认书》的，应当承担法律责任。竞得人拒绝签订《成交确认书》也不能对抗挂牌成交结果的法律效力。《成交确认书》应包括挂牌人与竞得人的名称，出让标的，成时间、地点、价款，以及双方签订《国有土地使用权出让合同》的时间、地点等内容。《成交确认书》对挂牌人和竞得人具有法律效力，挂牌人改变挂牌结果的，或者竞得人不按规定签订《国有土地使用权出让合同》、放弃竞得宗地的，应当承担法律责任。挂牌过程应当制作挂牌笔录。14\u3000签订出让合同，公布出让结果14．1\u3000签订《国有土地使用权出让合同》招标拍卖挂牌出让活动结束后，中标人、竞得人应按照《中标通知书》或《成交确认书》的约定，与出让人签订《国有土地使用权出让合同》。14．2\u3000中标人、竞得人支付的投标、竞买保证金，在中标或竞得后转作受让地块的定金。其他投标人、竞买人交纳的投标、竞买保证金，出让人应在招标拍卖挂牌活动结束后5个工作日内予以退还，不计利息。14．3\u3000公布出让结果招标拍卖挂牌活动结束后10 个工作日内，出让人应当将招标拍卖挂牌出让结果通过中国土地市场网以及土地有形市场等指定场所向社会公布。公布出让结果应当包括土地位置、面积、用途、开发程度、土地级别、容积率、出让年限、供地方式、受让人、成交价格和成交时间等内容。出让人公布出让结果，不得向受让人收取费用。15\u3000核发《建设用地批准书》，交付土地市、县国土资源管理部门向受让人核发《建设用地批准书》，并按照《国有土地使用权出让合同》、《建设用地批准书》确定的时间和条件将出让土地交付给受让人。16\u3000办理土地登记受让人按照《国有土地使用权出让合同》约定付清全部国有土地使用权出让金，依法申请办理土地登记，领取《国有土地使用证》，取得国有土地使用权。17\u3

In [49]:
len("一二三四444")

7

In [57]:
text = """（二）形势和要求。我国正处于深化经济体制改革和完善社会主义市场经济体制的攻坚期。现代市场经济是信用经济，建立健全社会信用体系，是整顿和规范市场经济秩序、改善市场信用环境、降低交易成本、防范经济风险的重要举措，是减少政府对经济的行政干预、完善社会主义市场经济体制的迫切要求。我国正处于加快转变发展方式、实现科学发展的战略机遇期。加快推进社会信用体系建设，是促进资源优化配置、扩大内需、促进产业结构优化升级的重要前提，是完善科学发展机制的迫切要求。我国正处于经济社会转型的关键期。利益主体更加多元化，各种社会矛盾凸显，社会组织形式及管理方式也在发生深刻变化。全面推进社会信用体系建设，是增强社会诚信、促进社会互信、减少社会矛盾的有效手段，是加强和创新社会治理、构建社会主义和谐社会的迫切要求。我国正处于在更大范围、更宽领域、更深层次上提高开放型经济水平的拓展期。经济全球化使我国对外开放程度不断提高，与其他国家和地区的经济社会交流更加密切。完善社会信用体系，是深化国际合作与交往，树立国际品牌和声誉，降低对外交易成本，提升国家软实力和国际影响力的必要条件，是推动建立客观、公正、合理、平衡的国际信用评级体系，适应全球化新形势，驾驭全球化新格局的迫切要求。（三）指导思想和目标原则。全面推动社会信用体系建设，必须坚持以邓小平理论、“三个代表”重要思想、科学发展观为指导，按照党的十八大、十八届三中全会和“十二五”规划纲要精神，以健全信用法律法规和标准体系、形成覆盖全社会的征信系统为基础，以推进政务诚信、商务诚信、社会诚信和司法公信建设为主要内容，以推进诚信文化建设、建立守信激励和失信惩戒机制为重点，以推进行业信用建设、地方信用建设和信用服务市场发展为支撑，以提高全社会诚信意识和信用水平、改善经济社会运行环境为目的，以人为本，在全社会广泛形成守信光荣、失信可耻的浓厚氛围，使诚实守信成为全民的自觉行为规范。社会信用体系建设的主要目标是：到2020年，社会信用基础性法律法规和标准体系基本建立，以信用信息资源共享为基础的覆盖全社会的征信系统基本建成，信用监管体制基本健全，信用服务市场体系比较完善，守信激励和失信惩戒机制全面发挥作用。政务诚信、商务诚信、社会诚信和司法公信建设取得明显进展，市场和社会满意度大幅提高。全社会诚信意识普遍增强，经济社会发展信用环境明显改善，经济社会秩序显著好转。社会信用体系建设的主要原则是：政府推动，社会共建。充分发挥政府的组织、引导、推动和示范作用。政府负责制定实施发展规划，健全法规和标准，培育和监管信用服务市场。注重发挥市场机制作用，协调并优化资源配置，鼓励和调动社会力量，广泛参与，共同推进，形成社会信用体系建设合力。健全法制，规范发展。逐步建立健全信用法律法规体系和信用标准体系，加强信用信息管理，规范信用服务体系发展，维护信用信息安全和信息主体权益。统筹规划，分步实施。针对社会信用体系建设的长期性、系统性和复杂性，强化顶层设计，立足当前，着眼长远，统筹全局，系统规划，有计划、分步骤地组织实施。重点突破，强化应用。选择重点领域和典型地区开展信用建设示范。积极推广信用产品的社会化应用，促进信用信息互联互通、协同共享，健全社会信用奖惩联动机制，营造诚实、自律、守信、互信的社会信用环境。二、推进重点领域诚信建设（一）加快推进政务诚信建设。政务诚信是社会信用体系建设的关键，各类政务行为主体的诚信水平，对其他社会主体诚信建设发挥着重要的表率和导向作用。坚持依法行政。将依法行政贯穿于决策、执行、监督和服务的全过程，全面推进政务公开，在保护国家信息安全、商业秘密和个人隐私的前提下，依法公开在行政管理中掌握的信用信息，建立有效的信息共享机制。切实提高政府工作效率和服务水平，转变政府职能。健全权力运行制约和监督体系，确保决策权、执行权、监督权既相互制约又相互协调。完善政府决策机制和程序，提高决策透明度。进一步推广重大决策事项公示和听证制度，拓宽公众参与政府决策的渠道，加强对权力运行的社会监督和约束，提升政府公信力，树立政府公开、公平、清廉的诚信形象。发挥政府诚信建设示范作用。各级人民政府首先要加强自身诚信建设，以政府的诚信施政，带动全社会诚信意识的树立和诚信水平的提高。在行政许可、政府采购、招标投标、劳动就业、社会保障、科研管理、干部选拔任用和管理监督、申请政府资金支持等领域，率先使用信用信息和信用产品，培育信用服务市场发展。加快政府守信践诺机制建设。严格履行政府向社会作出的承诺，把政务履约和守诺服务纳入政府绩效评价体系，把发展规划和政府工作报告关于经济社会发展目标落实情况以及为百姓办实事的践诺情况作为评价政府诚信水平的重要内容，推动各地区、各部门逐步建立健全政务和行政承诺考核制度。各级人民政府对依法作出的政策承诺和签订的各类合同要认真履约和兑现。要积极营造公平竞争、统一高效的市场环境，不得施行地方保护主义措施，如滥用行政权力封锁市场、包庇纵容行政区域内社会主体的违法违规和失信行为等。要支持统计部门依法统计、真实统计。政府举债要依法依规、规模适度、风险可控、程序透明。政府收支必须强化预算约束，提高透明度。加强和完善群众监督和舆论监督机制。完善政务诚信约束和问责机制。各级人民政府要自觉接受本级人大的法律监督和政协的民主监督。加大监察、审计等部门对行政行为的监督和审计力度。加强公务员诚信管理和教育。建立公务员诚信档案，依法依规将公务员个人有关事项报告、廉政记录、年度考核结果、相关违法违纪违约行为等信用信息纳入档案，将公务员诚信记录作为干部考核、任用和奖惩的重要依据。深入开展公务员诚信、守法和道德教育，加强法律知识和信用知识学习，编制公务员诚信手册，增强公务员法律和诚信意识，建立一支守法守信、高效廉洁的公务员队伍。（二）深入推进商务诚信建设。提高商务诚信水平是社会信用体系建设的重点，是商务关系有效维护、商务运行成本有效降低、营商环境有效改善的基本条件，是各类商务主体可持续发展的生存之本，也是各类经济活动高效开展的基础保障。生产领域信用建设。建立安全生产信用公告制度，完善安全生产承诺和安全生产不良信用记录及安全生产失信行为惩戒制度。以煤矿、非煤矿山、危险化学品、烟花爆竹、特种设备生产企业以及民用爆炸物品生产、销售企业和爆破企业或单位为重点，健全安全生产准入和退出信用审核机制，促进企业落实安全生产主体责任。以食品、药品、日用消费品、农产品和农业投入品为重点，加强各类生产经营主体生产和加工环节的信用管理，建立产品质量信用信息异地和部门间共享制度。推动建立质量信用征信系统，加快完善12365产品质量投诉举报咨询服务平台，建立质量诚信报告、失信黑名单披露、市场禁入和退出制度。流通领域信用建设。研究制定商贸流通领域企业信用信息征集共享制度，完善商贸流通企业信用评价基本规则和指标体系。推进批发零售、商贸物流、住宿餐饮及居民服务行业信用建设，开展企业信用分类管理。完善零售商与供应商信用合作模式。强化反垄断与反不正当竞争执法，加大对市场混淆行为、虚假宣传、商业欺诈、商业诋毁、商业贿赂等违法行为的查处力度，对典型案件、重大案件予以曝光，增加企业失信成本，促进诚信经营和公平竞争。逐步建立以商品条形码等标识为基础的全国商品流通追溯体系。加强检验检疫质量诚信体系建设。支持商贸服务企业信用融资，发展商业保理，规范预付消费行为。鼓励企业扩大信用销售，促进个人信用消费。推进对外经济贸易信用建设，进一步加强对外贸易、对外援助、对外投资合作等领域的信用信息管理、信用风险监测预警和企业信用等级分类管理。借助电子口岸管理平台，建立完善进出口企业信用评价体系、信用分类管理和联合监管制度。金融领域信用建设。创新金融信用产品，改善金融服务，维护金融消费者个人信息安全，保护金融消费者合法权益。加大对金融欺诈、恶意逃废银行债务、内幕交易、制售假保单、骗保骗赔、披露虚假信息、非法集资、逃套骗汇等金融失信行为的惩戒力度，规范金融市场秩序。加强金融信用信息基础设施建设，进一步扩大信用记录的覆盖面，强化金融业对守信者的激励作用和对失信者的约束作用。税务领域信用建设。建立跨部门信用信息共享机制。开展纳税人基础信息、各类交易信息、财产保有和转让信息以及纳税记录等涉税信息的交换、比对和应用工作。进一步完善纳税信用等级评定和发布制度，加强税务领域信用分类管理，发挥信用评定差异对纳税人的奖惩作用。建立税收违法黑名单制度。推进纳税信用与其他社会信用联动管理，提升纳税人税法遵从度。价格领域信用建设。指导企业和经营者加强价格自律，规范和引导经营者价格行为，实行经营者明码标价和收费公示制度，着力推行“明码实价”。督促经营者加强内部价格管理，根据经营者条件建立健全内部价格管理制度。完善经营者价格诚信制度，做好信息披露工作，推动实施奖惩制度。强化价格执法检查与反垄断执法，依法查处捏造和散布涨价信息、价格欺诈、价格垄断等价格失信行为，对典型案例予以公开曝光，规范市场价格秩序。工程建设领域信用建设。推进工程建设市场信用体系建设。加快工程建设市场信用法规制度建设，制定工程建设市场各方主体和从业人员信用标准。推进工程建设领域项目信息公开和诚信体系建设，依托政府网站，全面设立项目信息和信用信息公开共享专栏，集中公开工程建设项目信息和信用信息，推动建设全国性的综合检索平台，实现工程建设项目信息和信用信息公开共享的“一站式”综合检索服务。深入开展工程质量诚信建设。完善工程建设市场准入退出制度，加大对发生重大工程质量、安全责任事故或有其他重大失信行为的企业及负有责任的从业人员的惩戒力度。建立企业和从业人员信用评价结果与资质审批、执业资格注册、资质资格取消等审批审核事项的关联管理机制。建立科学、有效的建设领域从业人员信用评价机制和失信责任追溯制度，将肢解发包、转包、违法分包、拖欠工程款和农民工工资等列入失信责任追究范围。政府采购领域信用建设。加强政府采购信用管理，强化联动惩戒，保护政府采购当事人的合法权益。制定供应商、评审专家、政府采购代理机构以及相关从业人员的信用记录标准。依法建立政府采购供应商不良行为记录名单，对列入不良行为记录名单的供应商，在一定期限内禁止参加政府采购活动。完善政府采购市场的准入和退出机制，充分利用工商、税务、金融、检察等其他部门提供的信用信息，加强对政府采购当事人和相关人员的信用管理。加快建设全国统一的政府采购管理交易系统，提高政府采购活动透明度，实现信用信息的统一发布和共享。招标投标领域信用建设。扩大招标投标信用信息公开和共享范围，建立涵盖招标投标情况的信用评价指标和评价标准体系，健全招标投标信用信息公开和共享制度。进一步贯彻落实招标投标违法行为记录公告制度，推动完善奖惩联动机制。依托电子招标投标系统及其公共服务平台，实现招标投标和合同履行等信用信息的互联互通、实时交换和整合共享。鼓励市场主体运用基本信用信息和第三方信用评价结果，并将其作为投标人资格审查、评标、定标和合同签订的重要依据。交通运输领域信用建设。形成部门规章制度和地方性法规、地方政府规章相结合的交通运输信用法规体系。完善信用考核标准，实施分类考核监管。针对公路、铁路、水路、民航、管道等运输市场不同经营门类分别制定考核指标，加强信用考核评价监督管理，积极引导第三方机构参与信用考核评价，逐步建立交通运输管理机构与社会信用评价机构相结合，具有监督、申诉和复核机制的综合考核评价体系。将各类交通运输违法行为列入失信记录。鼓励和支持各单位在采购交通运输服务、招标投标、人员招聘等方面优先选择信用考核等级高的交通运输企业和从业人员。对失信企业和从业人员，要加强监管和惩戒，逐步建立跨地区、跨行业信用奖惩联动机制。电子商务领域信用建设。建立健全电子商务企业客户信用管理和交易信用评估制度，加强电子商务企业自身开发和销售信用产品的质量监督。推行电子商务主体身份标识制度，完善网店实名制。加强网店产品质量检查，严厉查处电子商务领域制假售假、传销活动、虚假广告、以次充好、服务违约等欺诈行为。打击内外勾结、伪造流量和商业信誉的行为，对失信主体建立行业限期禁入制度。促进电子商务信用信息与社会其他领域相关信息的交换和共享，推动电子商务与线下交易信用评价。完善电子商务信用服务保障制度，推动信用调查、信用评估、信用担保、信用保险、信用支付、商账管理等第三方信用服务和产品在电子商务中的推广应用。开展电子商务网站可信认证服务工作，推广应用网站可信标识，为电子商务用户识别假冒、钓鱼网站提供手段。统计领域信用建设。开展企业诚信统计承诺活动，营造诚实报数光荣、失信造假可耻的良好风气。完善统计诚信评价标准体系。建立健全企业统计诚信评价制度和统计从业人员诚信档案。加强执法检查，严厉查处统计领域的弄虚作假行为，建立统计失信行为通报和公开曝光制度。加大对统计失信企业的联合惩戒力度。将统计失信企业名单档案及其违法违规信息纳入金融、工商等行业和部门信用信息系统，将统计信用记录与企业融资、政府补贴、工商注册登记等直接挂钩，切实强化对统计失信行为的惩戒和制约。中介服务业信用建设。建立完善中介服务机构及其从业人员的信用记录和披露制度，并作为市场行政执法部门实施信用分类管理的重要依据。重点加强公证仲裁类、律师类、会计类、担保类、鉴证类、检验检测类、评估类、认证类、代理类、经纪类、职业介绍类、咨询类、交易类等机构信用分类管理，探索建立科学合理的评估指标体系、评估制度和工作机制。会展、广告领域信用建设。推动展会主办机构诚信办展，践行诚信服务公约，建立信用档案和违法违规单位信息披露制度，推广信用服务和产品的应用。加强广告业诚信建设，建立健全广告业信用分类管理制度，打击各类虚假广告，突出广告制作、传播环节各参与者责任，完善广告活动主体失信惩戒机制和严重失信淘汰机制。企业诚信管理制度建设。开展各行业企业诚信承诺活动，加大诚信企业示范宣传和典型失信案件曝光力度，引导企业增强社会责任感，在生产经营、财务管理和劳动用工管理等各环节中强化信用自律，改善商务信用生态环境。鼓励企业建立客户档案、开展客户诚信评价，将客户诚信交易记录纳入应收账款管理、信用销售授信额度计量，建立科学的企业信用管理流程，防范信用风险，提升企业综合竞争力。强化企业在发债、借款、担保等债权债务信用交易及生产经营活动中诚信履约。鼓励和支持有条件的企业设立信用管理师。鼓励企业建立内部职工诚信考核与评价制度。加强供水、供电、供热、燃气、电信、铁路、航空等关系人民群众日常生活行业企业的自身信用建设。（三）全面推进社会诚信建设。社会诚信是社会信用体系建设的基础，社会成员之间只有以诚相待、以信为本，才会形成和谐友爱的人际关系，才能促进社会文明进步，实现社会和谐稳定和长治久安。医药卫生和计划生育领域信用建设。加强医疗卫生机构信用管理和行业诚信作风建设。树立大医精诚的价值理念，坚持仁心仁术的执业操守。培育诚信执业、诚信采购、诚信诊疗、诚信收费、诚信医保理念，坚持合理检查、合理用药、合理治疗、合理收费等诚信医疗服务准则，全面建立药品价格、医疗服务价格公示制度，开展诚信医院、诚信药店创建活动，制定医疗机构和执业医师、药师、护士等医务人员信用评价指标标准，推进医院评审评价和医师定期考核，开展医务人员医德综合评价，惩戒收受贿赂、过度诊疗等违法和失信行为，建立诚信医疗服务体系。加快完善药品安全领域信用制度，建立药品研发、生产和流通企业信用档案。积极开展以“诚信至上，以质取胜”为主题的药品安全诚信承诺活动，切实提高药品安全信用监管水平，严厉打击制假贩假行为，保障人民群众用药安全有效。加强人口计生领域信用建设，开展人口和计划生育信用信息共享工作。社会保障领域信用建设。在救灾、救助、养老、社会保险、慈善、彩票等方面，建立全面的诚信制度，打击各类诈捐骗捐等失信行为。建立健全社会救助、保障性住房等民生政策实施中的申请、审核、退出等各环节的诚信制度，加强对申请相关民生政策的条件审核，强化对社会救助动态管理及保障房使用的监管，将失信和违规的个人纳入信用黑名单。构建居民家庭经济状况核对信息系统，建立和完善低收入家庭认定机制，确保社会救助、保障性住房等民生政策公平、公正和健康运行。建立健全社会保险诚信管理制度，加强社会保险经办管理，加强社会保险领域的劳动保障监督执法，规范参保缴费行为，加大对医保定点医院、定点药店、工伤保险协议医疗机构等社会保险协议服务机构及其工作人员、各类参保人员的违规、欺诈、骗保等行为的惩戒力度，防止和打击各种骗保行为。进一步完善社会保险基金管理制度，提高基金征收、管理、支付等各环节的透明度，推动社会保险诚信制度建设，规范参保缴费行为，确保社会保险基金的安全运行。劳动用工领域信用建设。进一步落实和完善企业劳动保障守法诚信制度，制定重大劳动保障违法行为社会公示办法。建立用人单位拖欠工资违法行为公示制度，健全用人单位劳动保障诚信等级评价办法。规范用工行为，加强对劳动合同履行和仲裁的管理，推动企业积极开展和谐劳动关系创建活动。加强劳动保障监督执法，加大对违法行为的打击力度。加强人力资源市场诚信建设，规范职业中介行为，打击各种黑中介、黑用工等违法失信行为。教育、科研领域信用建设。加强教师和科研人员诚信教育。开展教师诚信承诺活动，自觉接受广大学生、家长和社会各界的监督。发挥教师诚信执教、为人师表的影响作用。加强学生诚信教育，培养诚实守信良好习惯，为提高全民族诚信素质奠定基础。探索建立教育机构及其从业人员、教师和学生、科研机构和科技社团及科研人员的信用评价制度，将信用评价与考试招生、学籍管理、学历学位授予、科研项目立项、专业技术职务评聘、岗位聘用、评选表彰等挂钩，努力解决学历造假、论文抄袭、学术不端、考试招生作弊等问题。文化、体育、旅游领域信用建设。依托全国文化市场技术监管与公共服务平台，建立健全娱乐、演出、艺术品、网络文化等领域文化企业主体、从业人员以及文化产品的信用信息数据库；依法制定文化市场诚信管理措施，加强文化市场动态监管。制定职业体育从业人员诚信从业准则，建立职业体育从业人员、职业体育俱乐部和中介企业信用等级的第三方评估制度，推进相关信用信息记录和信用评级在参加或举办职业体育赛事、职业体育准入、转会等方面广泛运用。制定旅游从业人员诚信服务准则，建立旅游业消费者意见反馈和投诉记录与公开制度，建立旅行社、旅游景区和宾馆饭店信用等级第三方评估制度。知识产权领域信用建设。建立健全知识产权诚信管理制度，出台知识产权保护信用评价办法。重点打击侵犯知识产权和制售假冒伪劣商品行为，将知识产权侵权行为信息纳入失信记录，强化对盗版侵权等知识产权侵权失信行为的联合惩戒，提升全社会的知识产权保护意识。开展知识产权服务机构信用建设，探索建立各类知识产权服务标准化体系和诚信评价制度。环境保护和能源节约领域信用建设。推进国家环境监测、信息与统计能力建设，加强环保信用数据的采集和整理，实现环境保护工作业务协同和信息共享，完善环境信息公开目录。建立环境管理、监测信息公开制度。完善环评文件责任追究机制，建立环评机构及其从业人员、评估专家诚信档案数据库，强化对环评机构及其从业人员、评估专家的信用考核分类监管。建立企业对所排放污染物开展自行监测并公布污染物排放情况以及突发环境事件发生和处理情况制度。建立企业环境行为信用评价制度，定期发布评价结果，并组织开展动态分类管理，根据企业的信用等级予以相应的鼓励、警示或惩戒。完善企业环境行为信用信息共享机制，加强与银行、证券、保险、商务等部门的联动。加强国家能源利用数据统计、分析与信息上报能力建设。加强重点用能单位节能目标责任考核，定期公布考核结果，研究建立重点用能单位信用评价机制。强化对能源审计、节能评估和审查机构及其从业人员的信用评级和监管。研究开展节能服务公司信用评价工作，并逐步向全社会定期发布信用评级结果。加强对环资项目评审专家从业情况的信用考核管理。社会组织诚信建设。依托法人单位信息资源库，加快完善社会组织登记管理信息。健全社会组织信息公开制度，引导社会组织提升运作的公开性和透明度，规范社会组织信息公开行为。把诚信建设内容纳入各类社会组织章程，强化社会组织诚信自律，提高社会组织公信力。发挥行业协会（商会）在行业信用建设中的作用，加强会员诚信宣传教育和培训。自然人信用建设。突出自然人信用建设在社会信用体系建设中的基础性作用，依托国家人口信息资源库，建立完善自然人在经济社会活动中的信用记录，实现全国范围内自然人信用记录全覆盖。加强重点人群职业信用建设，建立公务员、企业法定代表人、律师、会计从业人员、注册会计师、统计从业人员、注册税务师、审计师、评估师、认证和检验检测从业人员、证券期货从业人员、上市公司高管人员、保险经纪人、医务人员、教师、科研人员、专利服务从业人员、项目经理、新闻媒体从业人员、导游、执业兽医等人员信用记录，推广使用职业信用报告，引导职业道德建设与行为规范。互联网应用及服务领域信用建设。大力推进网络诚信建设，培育依法办网、诚信用网理念，逐步落实网络实名制，完善网络信用建设的法律保障，大力推进网络信用监管机制建设。建立网络信用评价体系，对互联网企业的服务经营行为、上网人员的网上行为进行信用评估，记录信用等级。建立涵盖互联网企业、上网个人的网络信用档案，积极推进建立网络信用信息与社会其他领域相关信用信息的交换共享机制，大力推动网络信用信息在社会各领域推广应用。建立网络信用黑名单制度，将实施网络欺诈、造谣传谣、侵害他人合法权益等严重网络失信行为的企业、个人列入黑名单，对列入黑名单的主体采取网上行为限制、行业禁入等措施，通报相关部门并进行公开曝光。"""

In [58]:
len(text)

8913